<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 Análisis de sentimentos y textos</p>
<p style="margin: 0; text-align:right;">Máster Universitario en Ciencia de Datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 1: Procesamiento y análisis de información textual

En esta práctica revisaremos y aplicaremos los conocimientos aprendidos en los módulos del 1 al 2. Concretamente trataremos 3 temas.

<ul>
<li>1. Obtención de datos a partir de información textual
<li>2. Detección de tópicos
<li>3. Clasificación de textos
</ul>

El propósito de la práctica es descubrir rasgos característicos de las opiniones sobre restaurantes con las herramientas explicadas en los módulos del 1 al 2. Además, veremos si es posible clasificar automáticamente una opinión como positiva o negativa con métodos de machine learning. Utilizaremos el dataset <i>restaurants_reviews.csv</i>, extraído de una plataforma de expresión de opiniones. Este dataset contiene opiniones sobre restaurantes en inglés. El dataset se organiza en 10 columnas:

<b>business_id</b>: identificador del restaurant<br>
<b>date</b>: fecha de publicación de la opinión<br>
<b>review_id</b>: identificador de la opinión<br>
<b>stars</b>: calificación o valoración del restaurant en estrellas<br>
<b>text</b>: texto de la opinión<br>
<b>type</b>: tipo de texto<br>
<b>user_id</b>: identificador de usuario<br>
<b>cool, useful </b> y <b>funny</b>: Número de valoraciones que han realizado los usuarios de la plataforma para estos tres criterios.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Preparación del dataset

In [2]:
import pandas as pd
# Obrir el fitxer de comentaris:
df = pd.read_csv('restaurants_reviews.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,4,3,1
4,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


Para realizar la práctica, sólo necesitaremos los textos y las valoraciones en estrellas. Por tanto, eliminamos las columnas innecesarias y nos quedaremos solo con las columnas <b>stars</b> y <b>text</b>.

In [3]:
df.drop(['business_id', 'review_id', 'user_id', 'date', 'type', 'funny', 'cool', 'useful'], axis=1, inplace=True)
df

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,4,"Quiessence is, simply put, beautiful. Full wi..."
4,5,Drop what you're doing and drive here. After I...
...,...,...
6906,3,First visit...Had lunch here today - used my G...
6907,4,Should be called house of deliciousness!\r\n\r...
6908,4,I recently visited Olive and Ivy for business ...
6909,2,My nephew just moved to Scottsdale recently so...


Trabajaremos con las opiniones que tienen las calificaciones más altas y las más bajas. Las opiniones cuya calificación sea mayor que 3 serán etiquetadas con '1', mientras que etiquetaremos con '0' las opiniones que tengan una calificación menor a 3. Las etiquetas se asignan a la columna <b>sentiment</b>.

In [4]:
df = df[(df['stars'] > 3) | (df['stars'] < 3)]

df['sentiment'] = df['stars'].apply(lambda x : 1 if x > 3 else 0)


# Preprocesamiento

Antes de trabajar con los textos de las opiniones, hay que limpiarlos de caracteres como los saltos de línea (e.g: *Should be called house of deliciousness!\r\n\r*)

In [5]:
class CarriageReturnReplacer(object):
    """ Replaces \r\n expressions in a text.
    >>> replacer = CarriageReturnReplacer()
    >>> replacer.replace("\r\n\r\nAnyway, I can\'t wait to go back!")
    'Anyway, I can\'t wait to go back!'
    """
    
    def replace(self, text):
        s = text
        s = s.replace('\r\n', ' ')
        s = s.replace('\n\n', ' ') 
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ') 
        return s

newline_replacer = CarriageReturnReplacer()

In [6]:
df['text'] = df['text'].apply(newline_replacer.replace)

df

,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,4,"Quiessence is, simply put, beautiful. Full wi...",1
4,5,Drop what you're doing and drive here. After I...,1
...,...,...,...
6905,4,"Judging by some of the reviews, maybe I went o...",1
6907,4,Should be called house of deliciousness! I co...,1
6908,4,I recently visited Olive and Ivy for business ...,1
6909,2,My nephew just moved to Scottsdale recently so...,0


También es necesario eliminar los doble espacios:

In [7]:
import re

class ExtraSpacesReplacer(object):
    """ Replaces extra spaces in a text.
    >>> replacer = ExtraSpacesReplacer()
    >>> replacer.replace("and it was excellent.  The weather was perfect")
    'and it was excellent. The weather was perfect'
    """
    
    def replace(self, text):
        s = text
        s = re.sub('\s\s+', ' ', s)
        return s

spaces_replacer = ExtraSpacesReplacer()

In [9]:
df['text'] = df['text'].apply(spaces_replacer.replace)

# 1. Obtención de datos a partir de información textual (4 puntos)

## 1.1 Encontrar colocaciones (2 puntos)

Recordemos que las colocaciones son términos multipalabra, es decir, secuencias de palabras que, en conjunto, tienen un significado que difiere significativamente del significado de cada palabra individual (e.g. New York tiene un significado distinto del que se puede derivar de New y de York).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Calcula los mejores bigramas y trigramas de las opiniones. (1 punto)
</div>

In [8]:
# Para este apartado es necesario cargar las siguientes librerías:
import nltk
nltk.download('all')
from nltk import pos_tag, word_tokenize
from nltk.collocations import *
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    | 

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat

[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_tagset to
[nltk_data]    |     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_tagset is already up-to-date!
[nltk_data]    | Downloading package universal

In [10]:
#Importar la lista de stopwords en inglés de la libreria NLTK.
stopwords = nltk.corpus.stopwords.words('english')
#Añadir stopwords
stopwords = stopwords + ['unknown', 've', 'hadn', 'll', 'didn', 'isn', 'doesn', 'hasn' ]

A partir del comando help(nltk.collocations.BigramAssocMeasures) explora la clase BigramAssocMeasures del módulo nltk.metrics.association y revisa las definiciones de las métricas de Likelihood Ratio (likelihood_ratio) y de Pointwise Mutual Information (pmi) se explica en el capítulo 5 del libro Foundations of Statistical Natural Language Processing (Manning & Schutze).

In [11]:
help(nltk.collocations.BigramAssocMeasures)

Help on class BigramAssocMeasures in module nltk.metrics.association:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |  - n_ii counts ``(w1, w2)``, i.e. the bigram being scored
 |  - n_ix counts ``(w1, *)``
 |  - n_xi counts ``(*, w2)``
 |  - n_xx counts ``(*, *)``, i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |  

<i>Primer paso</i>: Obtener los tokens del texto de las opiniones. Etiqueta estos tokens por su PoS. (0.5 puntos)

In [12]:
#Creamos texto en minúscula que recoja todas las opiniones

opinions = " ".join(df['text']).lower()

opinions[:500]

"my wife took me here on my birthday for breakfast and it was excellent. the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. our waitress was excellent and our food arrived quickly on the semi-busy saturday morning. it looked like the place fills up pretty quickly so the earlier you get here the better. do yourself a favor and get their bloody mary. it was phenomenal and simply the best i've ever had. i'm pretty sure they only use ingredients from th"

In [13]:
#############################################
# SOLUCIÓN                                   #
#############################################
tokens = nltk.word_tokenize(opinions)
pos_tags = nltk.pos_tag(tokens)
pos_tags[:10]

[('my', 'PRP$'),
 ('wife', 'NN'),
 ('took', 'VBD'),
 ('me', 'PRP'),
 ('here', 'RB'),
 ('on', 'IN'),
 ('my', 'PRP$'),
 ('birthday', 'NN'),
 ('for', 'IN'),
 ('breakfast', 'NN')]

<i>Segundo paso</i>: Calcular los 1000 mejores bigramas y los 1000 mejores trigramas a partir de los tokens etiquetados (e.g. [(Basic, JJ), ...]) del texto. Utiliza la métrica PMI o Likehood Ratio. Tienes que indicar por qué has escogido una y no otra. (0.5 puntos)

<b>Atención</b>: De los 1000 bigramas y trigramas, elige a los que no comienzan ni terminan con una stopword.

Recordemos la clasificación de etiquetas PoS.

<b>Etiquetas PoS</b>

<ul>
<li>DT: Determinante</li>
<li>JJ: Adjetivo</li>
<li>NN: Nombre en singular</li>
<li>NNS: Nombre en plural</li>
<li>VBD: Verbo en pasado</li>
<li>VBG: Verbo en gerundio</li>
<li>MD: Verbo modal</li>
<li>IN: Preposición o conjunción subordinada</li>
<li>PRP: Pronombre</li>
<li>RB: Adverbio</li>
<li>RP: Partícula</li>    
<li>CC: Conjunción coordinada</li>
<li>CD: Numeral</li>
</ul>

In [19]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Cargamos las métricas para el cálculo de bigramas y trigramas:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# Cálculo de los bigramas y trigramas con la métrica PMI
finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
finder.apply_word_filter(lambda w: w.lower() in stopwords)  # Filtrar stopwords
finder.apply_ngram_filter(lambda w1, w2: w1.lower() in stopwords or w2.lower() in stopwords)  # Filtrar bigramas con stopwords
bigrams = finder.nbest(bigram_measures.pmi, 1000)

finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
finder.apply_word_filter(lambda w: w.lower() in stopwords)  # Filtrar stopwords
finder.apply_ngram_filter(lambda w1, w2, w3: w1.lower() in stopwords or w3.lower() in stopwords)  # Filtrar trigramas con stopwords
trigrams = finder.nbest(trigram_measures.pmi, 1000)

# Filtrar bigramas y trigramas que comienzan o terminan con stopwords
filtered_bigrams = [f"{w1} {w2}" for (w1, w2) in bigrams if not w1.lower() in stopwords and not w2.lower() in stopwords]
filtered_trigrams = [f"{w1} {w2} {w3}" for (w1, w2, w3) in trigrams if not w1.lower() in stopwords and not w3.lower() in stopwords]

# Imprimir los mejores bigramas y trigramas en formato de lista
print("BEST BIGRAMS:")
print(str(filtered_bigrams[:1000]))

print("\n\nBEST TRIGRAMS:")
print(str(filtered_trigrams[:1000]))


BEST BIGRAMS:
["'08 cakebread", "'eggless eggrolss", "'how 'bout", "'real food'/a", "'visual fluffers", '-50 gauges', '-elizabeth rozin', '-or- tinga', '............. yeppie', '/glenlivit /glenmorangie', '1/16 teaspoon', '1202 mohave', '1980s pseudo-asian', '1min 45sec', '2-4-1. woot', '4/ beach-head', '49er flapjacks', '4th-tier mid-90', '50¢ addtl', '65+ eyefucking', '=shiner bock', 'aa meeting-', 'abe frohman', 'administrative assistant', 'advent epicurean', 'afterglow punctuated', 'alain keller', 'alessandro marchesan', 'altering drugs', 'ambience=four estrellas', 'angello dorato', 'anita bryant', 'anxiously awaiting', 'aug 18.', 'authoritarian overlords', 'auto auction', 'avid youth', 'azucena tovar', 'bacon/bleu cheese/walnut', 'bahay kubo', 'band/art brut/we', 'bano squirting', 'bar/boxing ring/vintage', 'bartholomew novles', 'baton rouge', 'battling db', 'beach-head 2000', 'bellys aching', 'bengan bartha', 'bernie kantak', 'bhangra muzak', 'bifocal specs', 'big-mouth carp', 'bi

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong>  Detectar ngramas que cumplen el patrón sintáctico de un sintagma nominal (e.g: adjetivo + nombre en singular/plural y nombre + nombre) (0.5 puntos)
</div>

In [21]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Definir la expresión regular que busca patrones de sintagma nominal
noun_pattern = r"""
    NP: {<JJ>*<NN.*>+}
        {<NN.*>+<NN.*>+}
"""

# Crear un analizador de sintaxis que use la expresión regular definida anteriormente
chunk_parser = nltk.RegexpParser(noun_pattern)

# Analizar la lista de etiquetas de POS y extraer los sintagmas nominales

tree = chunk_parser.parse(pos_tags)
result = []
for subtree in tree.subtrees():
    if subtree.label() == 'NP':
        # Convertir el subárbol de sintagma nominal en una cadena de texto y agregarlo a la lista de resultados
        noun_phrase = " ".join(word for word, pos in subtree.leaves())
        result.append(noun_phrase)

print(result[:100]) 


['wife', 'birthday', 'breakfast', 'weather', 'grounds', 'absolute pleasure', 'waitress', 'food', 'semi-busy saturday morning', 'place', 'favor', 'bloody mary', 'i', 'pretty sure', 'ingredients', 'garden', 'order', 'everything', 'menu', 'i', 'white truffle', 'eggs', 'vegetable skillet', 'pieces', 'bread', 'toast', 'i', 'i', 'i', 'idea', 'people', 'bad reviews', 'place', 'everyone', 'something', 'own fault', 'many people', 'case', 'friend', 'i', 'pm', 'past sunday', 'i', 'sunday evening', 'thought', 'seat', 'girl', 'someone', 'waiter', 'drink orders', 'everyone', 'host', 'waiter', 'server', 'prices', 'orders', 'baked spaghetti calzone', 'beef', 'pizza', 'calzone', 'one', 'pizza', 'friend', 'pizza', 'calzone', 'calzone', 'sweetish sauce', 'sauce', 'part', 'pizza', 'door', 'everything', 'bad reviewers', 'things', 'bad reviewers', 'serious issues', 'gyro plate', 'rice', 'i', 'candy selection', 'quiessence', 'beautiful', 'full windows', 'earthy wooden walls', 'feeling', 'warmth', 'restaurant

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Detectar colocaciones con un modelo de detección de frases, con el módulo Phraser de Gensim. Entrena el modelo con todas las opiniones (0,5 puntos)
</div>

<i>Primer paso</i>: Convertir las opiniones en una lista de phrases. Las phrases no deben ser stopwords. Tampoco deben empezar ni acabar con una stopword. (0.5 puntos)

In [29]:
opinions_string = " ".join(df['text'])

opinion_sentences = opinions_string.split('. ')

opinion_sentences[:10]

['My wife took me here on my birthday for breakfast and it was excellent',
 'The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure',
 'Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning',
 'It looked like the place fills up pretty quickly so the earlier you get here the better',
 'Do yourself a favor and get their Bloody Mary',
 "It was phenomenal and simply the best I've ever had",
 "I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it",
 'It was amazing',
 'While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious',
 'It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete']

In [30]:
#############################################
# SOLUCIÓN                                 #
#############################################
import gensim
from gensim.models.phrases import Phrases, Phraser

# Preprocesar las oraciones eliminando stopwords y aplicando el modelo de detección de frases
preprocessed_sentences = []
for sentence in opinion_sentences:
    words = sentence.split()
    words = [word.lower() for word in words if word.lower() not in stopwords]
    if len(words) > 1:
        if words[0] not in stopwords and words[-1] not in stopwords:
            preprocessed_sentences.append(words)
            

# Entrenar el modelo de detección de frases utilizando las oraciones preprocesadas
phrases = Phrases(preprocessed_sentences, min_count=5, threshold=10)
phraser = Phraser(phrases)

# Aplicar el modelo de detección de frases a las oraciones para obtener las colocaciones detectadas
collocations = phraser[preprocessed_sentences]

# Mostrar las primeras 10 colocaciones detectadas por pantalla
for i in range(10):
    print(" ".join(preprocessed_sentences[i]))


wife took birthday breakfast excellent
weather perfect made sitting outside overlooking grounds absolute pleasure
waitress excellent food arrived quickly semi-busy saturday morning
looked like place fills pretty quickly earlier get better
favor get bloody mary
phenomenal simply best i've ever
i'm pretty sure use ingredients garden blend fresh order
everything menu looks excellent, white truffle scrambled eggs vegetable skillet tasty delicious
came 2 pieces griddled bread amazing absolutely made meal complete
best "toast" i've ever


## 1.2 Vectorizar palabras y términos (2 puntos)

Exploraremos la vectorización de palabras y términos con el método Word2Vec.

Recordemos que el paquete gensim implementa un método para entrenar modelos Word2Vec.

In [31]:
import gensim

In [33]:
# Quitar espacios del texto:
opinion_phrases_no_stopwords =  [palabra for palabra in tokens if palabra.lower() not in stopwords]
opinion_phrases_stripped_no_stopwords = [c.strip() for c in opinion_phrases_no_stopwords]
opinion_phrases_stripped_no_stopwords[:10]

['wife',
 'took',
 'birthday',
 'breakfast',
 'excellent',
 '.',
 'weather',
 'perfect',
 'made',
 'sitting']

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
Ejercicio:</strong> Obtener targets de las opiniones y sus aspectos utilizando el modelo word2vec (2 puntos)
</div>

<i>Primer paso</i>: Convertir las phrases de cada oración en un token. Lo haremos concatenando los tokens de la phrase con el caracter '_' (e.g: 'scrambled eggs' -> 'scrambled_eggs'). Entonces, en cada oración sustituimos los bigramas que son phrases por la forma tokenizada (e.g: I made scrambled eggs -> I made scrambled_eggs). De esta forma, las colocaciones formarán parte del vocabulario del modelo word2vec que generaremos.

In [34]:
from nltk.util import ngrams

collocation_phrases = [phrase for phrase in list(set(opinion_phrases_stripped_no_stopwords)) if ' ' in phrase]

def transform_sentence(sentence):
    transformed_sentence = sentence
    n_grams = list(ngrams(nltk.word_tokenize(sentence), 2))
    ngrams_t = [' '.join(gram) for gram in n_grams]
    for ngram in ngrams_t:
        if ngram in collocation_phrases:
            opt = ngram.replace(' ', '_')
            transformed_sentence = transformed_sentence.replace(ngram,opt)
    return transformed_sentence

opinion_sentences_transformed = [transform_sentence(os) for os in opinion_sentences]

opinion_sentences_transformed[:10]

['My wife took me here on my birthday for breakfast and it was excellent',
 'The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure',
 'Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning',
 'It looked like the place fills up pretty quickly so the earlier you get here the better',
 'Do yourself a favor and get their Bloody Mary',
 "It was phenomenal and simply the best I've ever had",
 "I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it",
 'It was amazing',
 'While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious',
 'It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete']

<i>Segundo paso</i>: crear una sentence stream donde todos los tokens de las oraciones están lematizados. Los tokens no pueden ser stopwords ni tener un stopword al inicio o al final. Para simplificar la tarea, consideramos que el lema de una colocación no cambia y su PoS es 'col'. (e.g: ['The guests like scrambled eggs', 'The rooms were dirty'] -> [['the', 'guest', 'like', 'scrambled_eggs], ['the', 'room', 'be ', 'dirty']]). (1 punto)

In [35]:
#############################################
# SOLUCIÓN                                  #
#############################################
from nltk.stem import WordNetLemmatizer
# Crear objeto para lematizar tokens
lemmatizer = WordNetLemmatizer()

def transform_sentence(sentence):
    transformed_tokens = []
    for token in word_tokenize(sentence):
        if token.lower() not in stopwords and token.isalpha() and len(token) > 1:
            # apply lemmatization
            transformed_token = lemmatizer.lemmatize(token)
            transformed_tokens.append(transformed_token)
    return transformed_tokens

def transform_sentences(sentences):
    transformed_sentences = []
    for sentence in sentences:
        transformed_sentence = transform_sentence(sentence)
        transformed_sentences.append(transformed_sentence)
    return transformed_sentences

transformed_sentences = transform_sentences(opinion_sentences_transformed)

for i in range(10):
    print(transformed_sentences[i])


['wife', 'took', 'birthday', 'breakfast', 'excellent']
['weather', 'perfect', 'made', 'sitting', 'outside', 'overlooking', 'ground', 'absolute', 'pleasure']
['waitress', 'excellent', 'food', 'arrived', 'quickly', 'Saturday', 'morning']
['looked', 'like', 'place', 'fill', 'pretty', 'quickly', 'earlier', 'get', 'better']
['favor', 'get', 'Bloody', 'Mary']
['phenomenal', 'simply', 'best', 'ever']
['pretty', 'sure', 'use', 'ingredient', 'garden', 'blend', 'fresh', 'order']
['amazing']
['EVERYTHING', 'menu', 'look', 'excellent', 'white', 'truffle', 'scrambled', 'egg', 'vegetable', 'skillet', 'tasty', 'delicious']
['came', 'piece', 'griddled', 'bread', 'amazing', 'absolutely', 'made', 'meal', 'complete']


<i>Tercer paso</i>: Crear un modelo word2vec de las opiniones lematizadas. El modelo debe llamarse w2v_opinions (0.5 puntos)

In [36]:
#############################################
# SOLUCIÓ                                   #
#############################################

# Creacion Modelo
w2v_opinions  =  gensim.models.Word2Vec(transformed_sentences, vector_size=100,
                 window=5, min_count=1, workers=4)
# guardar modelo
w2v_opinions.save('w2v_opinions.model')


<i>Cuarto paso</i>: A partir del vocabulario del modelo word2vec, selecciona posibles aspectos de la opinión (e.g: food) y lista términos semánticamente relacionados con estos aspectos según este modelo. (0.5 puntos)

Obtener el vocabulario:

In [37]:
vocabulary = list(w2v_opinions.wv.key_to_index.keys())

In [54]:
#############################################
# SOLUCIÓN                                   #
#############################################

# Seleccionamos posibles aspectos
aspect_food = 'food'
aspect_clean = 'clean'
aspect_service = 'service'
aspect_price = 'price'

# Obtener términos semánticamente relacionados con los aspectos
related_terms_food = w2v_opinions.wv.most_similar(aspect_food, topn=10)
related_terms_clean = w2v_opinions.wv.most_similar(aspect_clean, topn=10)
related_terms_service = w2v_opinions.wv.most_similar(aspect_service, topn=10)
related_terms_price = w2v_opinions.wv.most_similar(aspect_price, topn=10)


In [55]:
#############################################
# SOLUCIÓN                                   #
#############################################

#Imprimimos ejemplos de aspectos cercanos semánticamente al target según el modelo Word2Vec que el alumno puede 
#encontrar en el dataframe anterior. Entre estos ejemplos está, por ejemplo, 'atmosphere' que tiene un valor de
#similitud semántica alto. El ejercicio consiste en que el alumno elija los ejemplos que considere pertinentes.

# Imprimir términos relacionados
print(f"\nTérminos relacionados con '{aspect_food}':")
for term, similarity in related_terms_food:
    print(f"{term} ({similarity:.2f})")
    
# Imprimir términos relacionados
print(f"\nTérminos relacionados con '{aspect_clean}':")
for term, similarity in related_terms_clean:
    print(f"{term} ({similarity:.2f})")
    
# Imprimir términos relacionados
print(f"\nTérminos relacionados con '{aspect_service}':")
for term, similarity in related_terms_service:
    print(f"{term} ({similarity:.2f})")
    
# Imprimir términos relacionados
print(f"\nTérminos relacionados con '{aspect_price}':")
for term, similarity in related_terms_price:
    print(f"{term} ({similarity:.2f})")


Términos relacionados con 'food':
service (0.95)
Service (0.94)
consistently (0.94)
Food (0.93)
ambiance (0.93)
Always (0.92)
atmosphere (0.92)
Overall (0.92)
always (0.92)
pretty (0.92)

Términos relacionados con 'clean':
competent (0.99)
polite (0.98)
welcoming (0.98)
courteous (0.98)
helpful (0.98)
prompt (0.98)
efficient (0.98)
comfortable (0.98)
smiling (0.98)
pleasant (0.98)

Términos relacionados con 'service':
Service (0.97)
atmosphere (0.96)
food (0.95)
ambiance (0.94)
consistently (0.94)
always (0.93)
efficient (0.93)
staff (0.92)
fast (0.92)
quick (0.92)

Términos relacionados con 'price':
reasonable (0.97)
Good (0.94)
decent (0.94)
reasonably (0.93)
pretty (0.93)
priced (0.93)
cheap (0.93)
selection (0.93)
value (0.93)
Great (0.93)


# 2. Detección de temas (4 puntos)

En estos apartados exploraremos cúales son los tópicos tratados en las opiniones.

## 2.1 Exploración de los temas con WordNet (2 puntos)

En este apartado accederemos a Wordnet a través de la librería nltk.

In [44]:
from nltk.corpus import wordnet as wn

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Comprueba si, según Wordnet, existen aspectos que están alejados semánticamente del sentido del target, aunque en el modelo word2vec sean similares. Compruébalo calculando la similitud de Wu and Palmer entre el sentido de wordnet 'restaurant.n.01' y algunos de sus aspectos. (1 punto)
</div>

In [64]:
#############################################
# SOLUCIÓN                                   #
#############################################

# Definir el synset de destino para restaurant.n.01
target_synset = wn.synset('restaurant.n.01')

# Definir los synsets de aspecto
aspect_synsets = [
    wn.synset('food.n.01'),
    wn.synset('clean.n.01'),
    wn.synset('service.n.01'),
    wn.synset('price.n.01')
]

# Calcule la similitud de Wu-Palmer entre el objetivo y cada synset de aspecto
for synset in aspect_synsets:
    similarity = target_synset.path_similarity(synset)
    print(f'Similitud WUP entre {target_synset.name()} y {synset.name()}: {similarity}')


Similitud WUP entre restaurant.n.01 y food.n.01: 0.1
Similitud WUP entre restaurant.n.01 y clean_and_jerk.n.01: 0.05
Similitud WUP entre restaurant.n.01 y service.n.01: 0.06666666666666667
Similitud WUP entre restaurant.n.01 y monetary_value.n.01: 0.07142857142857142


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Lista los términos monopalabra del vocabulario de word2vec que no están en Wordnet. Los términos deben ser nombres o adjetivos. ¿Creeis que estos términos son importantes para poder realizar un buen análisis de sentimientos? (1 punto)

In [66]:
#############################################
# SOLUCIÓN                                  #
#############################################

# Cargue el modelo word2vec
model = gensim.models.Word2Vec.load('w2v_opinions.model')

# Obtener el vocabulario del modelo.
vocab = set(model.wv.index_to_key)

# Obtenga un conjunto de sustantivos y adjetivos de WordNet
wordnet_nouns = set(list(wn.all_synsets('n')))
wordnet_adjectives = set(list(wn.all_synsets('a')))
wordnet_nouns_and_adjectives = wordnet_nouns.union(wordnet_adjectives)

#Encuentra términos de monopalabras en el vocabulario del modelo que no están en WordNet
missing_terms = []
for term in vocab:
    if (len(term.split('_')) == 1) and (term not in wordnet_nouns_and_adjectives):
        missing_terms.append(term)

# Imprime los términos que faltan
print(f"Primeros 10 términos monopalabras no presentes en WordNet: {missing_terms[:10]}")



Primeros 10 términos monopalabras no presentes en WordNet: ['listed', 'lopsided', 'borrowed', 'Highly', 'replacement', 'proposition', 'Oddity', 'Europa', 'strongbow', 'invisible']


## 2.2 LDA (2 puntos)

Recordad que en el notebook del módulo 1 hemos visto la aplicación del método LDA para extraer temas de documentos.

<i>Primer paso</i>: Convertir las opiniones transformadas (opinion_sentences_transformed) en listas de nombres y colocaciones. Esto es necesario ya que los nombres y las colocaciones expresan los temas de las opiniones (e.g: [['wife have breakfast sitting_outside'], [' 'waitress', 'served', 'ceviche']...] -> [['wife', 'breakfast', 'sitting_outside'], ['waitress', 'ceviche']]

In [67]:
def get_noun_and_collocation(sentence):
    nouns_and_collocations = []
    noun_tags = ['NN', 'NNS']
    tokens_pos_tagged = pos_tag(word_tokenize(sentence))
    for tpos in tokens_pos_tagged:
        if '_' in tpos[0]:
            nouns_and_collocations.append(tpos[0])
        elif tpos[1] in noun_tags:
            nouns_and_collocations.append(tpos[0])
    return nouns_and_collocations
            
noun_and_collocation_stream = [get_noun_and_collocation(opinion) for opinion in opinion_sentences_transformed]

noun_and_collocation_stream[:5]


[['wife', 'birthday', 'breakfast'],
 ['weather', 'grounds', 'pleasure'],
 ['waitress', 'food', 'morning'],
 ['place'],
 ['favor']]

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Extrae temas a partir de las listas de nombres y colocaciones de cada oración transformada. Experimenta con el parámetro num_topics hasta encontrar un conjunto de temas informativos, asignando nombres a los temas encontrados. (2 puntos)
</div>

In [75]:
#############################################
# SOLUCIÓN                                 #
#############################################

from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Crear el diccionario y el corpus de palabras
dictionary = Dictionary(noun_and_collocation_stream)
corpus = [dictionary.doc2bow(opinion) for opinion in noun_and_collocation_stream]

# Entrenar el modelo de LDA con diferentes valores de num_topics
for num_topics in [5, 10, 15, 20]:
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
    print(f"Modelo de LDA con {num_topics} temas:")
    for topic in lda_model.show_topics(num_topics=num_topics, formatted=False):
        print(f"Tema {topic[0]}: {[t[0] for t in topic[1]]}")
    print("="*125)

Modelo de LDA con 5 temas:
Tema 0: ['night', 'bar', 'pizza', 'hour', 'table', 'places', 'drink', 'way', 'beer', 'years']
Tema 1: ['food', 'place', 'service', 'lunch', 'burger', 'location', 'prices', 'restaurant', 'quality', 'lot']
Tema 2: ['order', 'sauce', 'chicken', 'cheese', 'side', 'salad', 'rice', 'plate', 'flavor', 'beef']
Tema 3: ['time', 'experience', 'people', 'drinks', 'staff', 'things', 'wine', 'family', 'home', 'reviews']
Tema 4: ['menu', 'service', '..', 'restaurant', 'meal', 'times', 'i', 'bread', 'anything', 'stars']
Modelo de LDA con 10 temas:
Tema 0: ['chicken', 'burger', 'sauce', 'fries', 'meat', 'lot', 'bread', 'beef', 'nothing', 'steak']
Tema 1: ['restaurant', 'night', 'dinner', 'experience', 'i', 'course', 'reviews', 'house', 'wife', 'room']
Tema 2: ['food', 'time', 'people', 'location', 'minutes', 'wait', 'family', 'order', 'couple', 'door']
Tema 3: ['staff', 'bar', 'hour', 'drinks', 'table', 'bit', 'side', 'items', 'dishes', 'rice']
Tema 4: ['order', 'times', 'so

En el primer modelo de LDA con 5 temas, se podría asignar los siguientes nombres a cada tema:

* Tema 0: Vida nocturna y bares
* Tema 1: Comida y precios
* Tema 2: Comida para llevar y pedidos
* Tema 3: Experiencia y servicio
* Tema 4: Menú y restaurantes

Para el segundo modelo de LDA con 10 temas, se podría asignar los siguientes nombres a cada tema:

* Tema 0: Comida rápida y hamburguesas
* Tema 1: Cena y experiencias en el restaurante
* Tema 2: Espera y tiempo de servicio
* Tema 3: Personal y bebidas
* Tema 4: Pedidos y decoración
* Tema 5: Comida y calidad
* Tema 6: Platos principales y especialidades
* Tema 7: Pizza y sándwiches
* Tema 8: Servicio y bebidas
* Tema 9: Almuerzo y precios

Para el tercer modelo de LDA con 15 temas, se podría asignar los siguientes nombres a cada tema:

* Tema 0: Menú y precios
* Tema 1: Algo diferente y divertido
* Tema 2: Mariscos y platos especiales
* Tema 3: Comida y opciones
* Tema 4: Almuerzo y tacos
* Tema 5: Papas fritas y café
* Tema 6: Ambiente y otros detalles
* Tema 7: Sabor y comida rápida
* Tema 8: Bebidas y aperitivos
* Tema 9: Servicio y vino
* Tema 10: Comida y calidad
* Tema 11: Tiempo y lugar
* Tema 12: Pollo y bebidas
* Tema 13: Cena y experiencia
* Tema 14: Sushi y carne de cerdo

Para el ultimo modelo de LDA con 20 temas, se podría asignar los siguientes nombres a cada tema:

* Tema 0: Personal y ambiente del establecimiento
* Tema 1: Servicio y calidad de los alimentos
* Tema 2: Variedad y opciones de comida
* Tema 3: Ambiente y problemas en el establecimiento
* Tema 4: Menú y opciones de comida
* Tema 5: Salsas y guarniciones
* Tema 6: Comida y ambiente del establecimiento
* Tema 7: Tiempo de espera y calidad del establecimiento
* Tema 8: Guarniciones y propietario del establecimiento
* Tema 9: Ambiente y público del establecimiento
* Tema 10: Ensaladas y opciones vegetarianas
* Tema 11: Precios y opciones de comida
* Tema 12: Comida y servicio
* Tema 13: Ambiente y calidad del establecimiento
* Tema 14: Carnes y opciones de comida
* Tema 15: Comida y ambiente del establecimiento
* Tema 16: Experiencia culinaria y variedad de opciones
* Tema 17: Ordenar y calidad de la comida
* Tema 18: Comida y bebida
* Tema 19: Servicio y popularidad del establecimiento.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Ejercicio opcional:</strong> Si consideramos también los verbos, ¿mejoraríamos los resultados? Justifica la respuesta mostrando LDA si consideramos los verbos, además de los nombres y colocaciones.
</div>

In [88]:
!pip install -U scikit-learn

In [89]:
#############################################
# SOLUCIÓN                                 #
#############################################
from gensim import corpora

# función para extraer nombres, verbos y colocaciones de una oración
def get_noun_verb_and_collocation(sentence):
    noun_verb_and_collocations = []
    noun_tags = ['NN', 'NNS']
    verb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    tokens_pos_tagged = pos_tag(word_tokenize(sentence))
    for i in range(len(tokens_pos_tagged)-1):
        if '_' in tokens_pos_tagged[i][0]:
            noun_verb_and_collocations.append(tokens_pos_tagged[i][0])
        elif tokens_pos_tagged[i][1] in noun_tags and tokens_pos_tagged[i+1][1] in verb_tags:
            noun_verb_and_collocations.append(tokens_pos_tagged[i][0])
        elif tokens_pos_tagged[i][1] in verb_tags and tokens_pos_tagged[i+1][1] in noun_tags:
            noun_verb_and_collocations.append(tokens_pos_tagged[i][0])
        elif '_' in tokens_pos_tagged[i][0] and tokens_pos_tagged[i+1][1] in verb_tags:
            noun_verb_and_collocations.append(tokens_pos_tagged[i][0])
        elif '_' in tokens_pos_tagged[i][0] and tokens_pos_tagged[i+1][1] in noun_tags:
            noun_verb_and_collocations.append(tokens_pos_tagged[i][0])
    return noun_verb_and_collocations


In [80]:
# extraer nombres, verbos y colocaciones de cada oración transformada
noun_verb_and_collocation_stream = [get_noun_verb_and_collocation(opinion) for opinion in opinion_sentences_transformed]

In [92]:
lda_model.show_topic(1, topn=10)

[('place', 0.1756753),
 ('service', 0.09894681),
 ('menu', 0.04671423),
 ('restaurant', 0.03467527),
 ('pizza', 0.022971252),
 ('atmosphere', 0.022602575),
 ('everything', 0.020159384),
 ('waitress', 0.019082388),
 ('take', 0.017288648),
 ('meal', 0.013537677)]

In [96]:
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models.ldamodel import LdaModel
from gensim import corpora

# crear diccionario y corpus
dictionary = corpora.Dictionary(noun_verb_and_collocation_stream)
corpus = [dictionary.doc2bow(opinion) for opinion in noun_verb_and_collocation_stream]

vocab = list(dictionary.values())
term_frequency = list(dictionary.dfs.values())

# entrenar el modelo LDA
num_topics = 5
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# mostrar los temas encontrados
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# visualizar el modelo LDA
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis)


(0, '0.031*"is" + 0.022*"atmosphere" + 0.021*"friend" + 0.020*"everything" + 0.014*"decor" + 0.013*"bar" + 0.013*"friends" + 0.013*"tacos" + 0.012*"ordered" + 0.012*"dining"')
(1, '0.173*"food" + 0.095*"service" + 0.053*"was" + 0.035*"people" + 0.022*"pizza" + 0.018*"drinks" + 0.018*"portions" + 0.015*"get" + 0.013*"everyone" + 0.013*"meal"')
(2, '0.022*"time" + 0.022*"fries" + 0.019*"waitress" + 0.019*"are" + 0.017*"dishes" + 0.017*"Food" + 0.015*"were" + 0.015*"sandwich" + 0.014*"dish" + 0.013*"has"')
(3, '0.057*"staff" + 0.050*"i" + 0.034*"restaurant" + 0.033*"server" + 0.027*"sauce" + 0.026*"prices" + 0.020*"husband" + 0.020*"bread" + 0.019*"location" + 0.018*"Everything"')
(4, '0.185*"place" + 0.050*"had" + 0.049*"menu" + 0.031*"have" + 0.026*"chicken" + 0.022*"\'s" + 0.021*"salad" + 0.018*"take" + 0.018*"waiter" + 0.016*"fried"')


In [91]:
#############################################
# SOLUCIÓN                                  #
#############################################

print("La incorporación de verbos mejora o no los resultados?")

La incorporación de verbos mejora o no los resultados?


En general, la incorporación de verbos puede ayudar a mejorar los resultados de LDA, ya que los verbos pueden proporcionar información adicional sobre las acciones y las relaciones entre los elementos en el texto. En este caso, parece que la incorporación de verbos ha mejorado la separación de los temas, y algunos de los temas parecen más coherentes y relevantes que en la versión anterior sin verbos.

# 3. Clasificación (2 puntos)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Crea un clasificador automático de opiniones positivas y negativas. (0.75 puntos)
</div>

<i>Primer paso</i>: realizamos dos listas. Una con los textos y otra con las etiquetas de valoración (0 y 1)

In [97]:
#############################################
# SOLUCIÓN                                  #
#############################################

opinions = df['text'].to_list()
labels = df['sentiment'].to_list()

<i>Segundo paso</i>: Vectorizamos las opiniones con un vectorizador tf.idf. Usad 'word' como analyzer (0.25 punts)

In [100]:
#############################################
# SOLUCIÓN                                 #
#############################################
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word')
opinions_vectorized = vectorizer.fit_transform(opinions)
print(opinions_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


<i>Tercer paso</i>: Preparamos el corpus de entrenamiento y evaluación (0.25 puntos)

In [101]:
#############################################
# SOLUCIÓN                                  #
#############################################
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(opinions_vectorized, labels, test_size=0.2, random_state=42)


<i>Cuarto paso</i>: Entrenar al clasificador con Logistic Regression

In [102]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
log_model = classifier.fit(X=X_train, y=y_train)

Quinto paso<i>: Utilizar el modelo entrenado para predecir la categoría 1 (positivo) o 0 (negativo) de las opiniones del conjunto de test y mostrar las palabras más informativas para cada categoría. (0.25 puntos)

In [105]:
#############################################
# SOLUCIÓN                                  #
#############################################
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
feature_names = vectorizer.get_feature_names()
coef = classifier.coef_[0]
top_positive_words = [feature_names[i] for i in coef.argsort()[-10:]]
top_negative_words = [feature_names[i] for i in coef.argsort()[:10]]

print("Palabras más informativas para la categoría positiva:", top_positive_words)
print("Palabras más informativas para la categoría negativa:", top_negative_words)
print("Accuracy:", accuracy)

Palabras más informativas para la categoría positiva: ['definitely', 'awesome', 'and', 'excellent', 'best', 'amazing', 'love', 'good', 'delicious', 'great']
Palabras más informativas para la categoría negativa: ['not', 'no', 'wasn', 'ok', 'bland', 'just', 'horrible', 'was', 'better', 'bad']
Accuracy: 0.8949615713065756


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercicio:</strong> Muestra sobre qué aspectos se hacen valoraciones negativas. (0.75 puntos)
</div>

<i>Primer paso<i>: Elige dos palabras más informativas de la categoría 0 y toma un conjunto de opiniones en las que aparezcan estas palabras. Preprocesa las opiniones quitando los caracteres de salto de línea (0.25 puntos)

In [106]:
#############################################
# SOLUCIÓN                                  # 
#############################################
# Elegimos dos palabras más informativas para la categoría negativa, por ejemplo "terrible" y "horrible"
selected_words = ["horrible", "bad"]

# Tomamos un conjunto de opiniones en las que aparezcan estas palabras
selected_opinions = []
for i in range(len(opinions)):
    opinion = opinions[i].replace('\n', '')
    if all(word in opinion for word in selected_words):
        selected_opinions.append(opinion)

# Preprocesamos las opiniones quitando los caracteres de salto de línea
selected_opinions = [opinion.replace('\n', '') for opinion in selected_opinions]


<i>Segundo paso<i>: Utiliza el diccionario de opiniones (archivo AFINN-111) para extraer la polaridad de cada opinión como la media de los valores de las opinion words del texto. (0.25 puntos)

In [107]:
#############################################
# SOLUCIÓN                                  #
#############################################

#Creamos una lista de opinion words y pesos a partir del diccionario AFINN
opinion_words_file = 'AFINN-111.txt'

opinion_words = {}
with open(opinion_words_file, 'r') as f:
    for line in f:
        word, score = line.split('\t')
        opinion_words[word] = int(score)

opinions_polarity = []
for opinion in selected_opinions:
    opinion_score = sum(opinion_words.get(word.lower(), 0) for word in opinion.split())
    opinions_polarity.append(opinion_score)

# Normalizamos la polaridad entre -1 y 1
max_score = max(abs(score) for score in opinions_polarity)
opinions_polarity = [score / max_score for score in opinions_polarity]



Tercer paso: Selecciona opiniones con polaridad negativa que ejemplifiquen los aspectos peor valorados. Comenta cuáles son estos aspectos (0.25 puntos)

In [108]:
#############################################
# SOLUCIÓN                                  #
#############################################
selected_opinions_neg = []
for i in range(len(opinions_polarity)):
    if opinions_polarity[i] < 0:
        selected_opinions_neg.append(selected_opinions[i])

# Mostrar las opiniones seleccionadas
print("Opiniones seleccionadas con polaridad negativa:")
for opinion in selected_opinions_neg:
    print("\n- ", opinion)


Opiniones seleccionadas con polaridad negativa:

-  I've eaten here many times, but none as bad as last night. Service was excellent, and highly attentive. Food, absolutely horrible. My expectation was they would serve a steak on par with their seafood. After all, they were charging 39 bucks for a ribeye. What I was hoping for was a 1- 1-1/2' thick steak, cooked Pittsburgh style as I had ordered. What I got a a 3/4 in thick piece of meat that was mostly fat, gristle, and in no way resembled Pittsburgh Style. Salad, similar to something you could get at Chick Filet Veggies, blah. Bread basket, ample, but day old, and if not, it certainly wasn't fresh. In addition to bad food, we were crammed into a small room where we were nuts to butts with 6 other tables, listening to conversations ranging from someone's recent bout with pinkeye, and another couple who elected to speak entirely in French, until the waiter showed up, then it was like they turned off the French switch and suddenly began

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Obtén los resultados de las métricas de evaluación del clasificador basado en regresión logística. Obtén también los resultados de las métricas de evaluación cuando el modelo del clasificador es diferente. Por ejemplo, un modelo SVM (0.25 puntos)
</div>

In [110]:
#############################################
# SOLUCIÓN MODELO REGRESIÓN LOGÍSTICA              #
#############################################
from sklearn.metrics import classification_report, confusion_matrix

# Obtenemos las predicciones del modelo en el conjunto de test
y_pred = classifier.predict(X_test)

# Mostramos la matriz de confusión
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

# Mostramos el reporte de clasificación con las métricas
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Matriz de confusión:
 [[ 98 115]
 [  8 950]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.92      0.46      0.61       213
           1       0.89      0.99      0.94       958

    accuracy                           0.89      1171
   macro avg       0.91      0.73      0.78      1171
weighted avg       0.90      0.89      0.88      1171



In [111]:
#############################################
# SOLUCIÓN MODELO SVM                       #
#############################################
from sklearn.svm import SVC

# Creamos un modelo SVM
svm = SVC(kernel='linear', random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
svm.fit(X_train, y_train)

# Obtenemos las predicciones del modelo en el conjunto de test
y_pred_svm = svm.predict(X_test)

# Mostramos la matriz de confusión
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_svm))

# Mostramos el reporte de clasificación con las métricas
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_svm))


Matriz de confusión:
 [[143  70]
 [ 23 935]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      0.67      0.75       213
           1       0.93      0.98      0.95       958

    accuracy                           0.92      1171
   macro avg       0.90      0.82      0.85      1171
weighted avg       0.92      0.92      0.92      1171



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Compara los dos modelos en función de estas métricas de evaluación. (0.25 puntos)
</div>

In [112]:
#############################################
# SOLUCIÓN                                  #
#############################################

print("Decir cuál de los dos modelos tiene mejores valores de precision recall y f1.\nComentar si las diferencias son significativas y apuntar las causas por las cuales un modelo da mejores resultados que el otro.")

Decir cuál de los dos modelos tiene mejores valores de precision recall y f1.
Comentar si las diferencias son significativas y apuntar las causas por las cuales un modelo da mejores resultados que el otro.


Ambos modelos presentan una precisión alta para la clase 1 (comentarios positivos), con valores superiores al 0.89. Sin embargo, en cuanto a la precisión para la clase 0 (comentarios negativos), el modelo de Regresión Logística presenta un valor de 0.92, mientras que el modelo SVM presenta un valor de 0.86.

En cuanto a la medida de recall, el modelo SVM presenta un valor ligeramente superior para ambas clases (0.67 para la clase 0 y 0.98 para la clase 1) en comparación con el modelo de Regresión Logística (0.46 para la clase 0 y 0.99 para la clase 1).

En cuanto al F1-score, el modelo SVM presenta un valor superior en general (0.75 para la clase 0 y 0.95 para la clase 1) en comparación con el modelo de Regresión Logística (0.61 para la clase 0 y 0.94 para la clase 1).

En conclusión, el modelo SVM presenta un rendimiento ligeramente superior en general en comparación con el modelo de Regresión Logística en este conjunto de datos y métricas de evaluación.